In [ ]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import torch
from notebooks.experiment_robust import load_robust_experiment
from notebooks.experiment_torch import load_torchvision_experiment

model, dl_train, dl_eval = load_robust_experiment("Standard", "cifar10")
# model, dl_train, dl_eval = load_torchvision_experiment("vgg16")

In [ ]:
class CosineSimilarity(torch.nn.Module):
    def forward(self, v1: torch.Tensor, v2: torch.Tensor) -> torch.Tensor:
        # we want to maximize this loss
        v1 = v1.view(v1.size(0), -1)
        v2 = v2.view(v2.size(0), -1)
        return torch.cosine_similarity(v1, v2, dim=1).mean()

In [ ]:
from ulib.attack import StopCriteria
from ulib.attacks.imi_uap import IMI_UAP
from ulib.pert_module import PertModule
import torchattacks

stop = StopCriteria(max_epochs=10, max_time=600)
torch_attack = torchattacks.FFGSM(model)
pert_model = PertModule(model, data_shape=dl_train.data.shape[1:], eps=8 / 255)
optimizer = torch.optim.Adam(pert_model.parameters(), lr=0.01, maximize=True)
criterion = CosineSimilarity()
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=1, eta_min=5e-5)
grad_scaler = torch.GradScaler(device=pert_model.device.type)

attack = IMI_UAP(
    pert_model=pert_model,
    optimizer=optimizer,
    inner_attack=torch_attack,
    criterion=criterion,
    scheduler=scheduler,
    grad_scaler=grad_scaler,
)

pert = attack.fit(dl_train, dl_eval, stop)

In [ ]:
attack.close()

In [ ]:
from ulib import eval

eval.full_analysis(pert_model, dl_eval)